# RandomForestClassifier

Faydalandığımız kaynak
- https://www.kaggle.com/code/akshitrai/chatbot-jarvis
- https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html

In [ ]:
import os
import json
import numpy as np 
import string
from unicode_tr import unicode_tr
from nltk.corpus import stopwords
from snowballstemmer import TurkishStemmer
from nltk import ngrams
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from time import time


In [ ]:
path = r'C:\Users\alierenekinci\Desktop\Project\ThreadTeknofest\veri'

def yukle():
    liste = os.listdir(path)
    final_path = path + "\\" + liste[-1]
    print(final_path)
    with open(final_path, encoding='utf-8') as f:
        veri = json.load(f)
    return veri

In [ ]:
df_ = yukle()

In [ ]:
questions, responses = [],[]


for veri_ in df_:
    for i, j in enumerate(df_[veri_]):
        for k,l in enumerate(j["patterns"]):
            questions.append(l)
            responses.append(df_[veri_][i]["responses"][0])
        # for k,l in enumerate(j["responses"]):
        #     responses.append(l)

In [ ]:
df = pd.DataFrame([questions,responses]).T

In [ ]:
df

In [ ]:
# Define a pipeline combining a text feature extractor with a simple classifier
pipeline = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("tfidf", TfidfTransformer()),
        ("clf", RandomForestClassifier()),
    ]
)

# Parameters to use for grid search. Uncommenting more parameters will give
# better exploring power but will increase processing time in a combinatorial
# way
parameters = {
    "vect__max_df": (0.5,),
    'vect__max_features': (10000,),
    # "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (False,),
    'tfidf__norm': ('l2',),
    "clf__n_estimators": (100,),
    "clf__criterion" : ("gini",),
    # "clf__max_depth" : (1, 2, 5, 7),
    "clf__min_samples_leaf" : (1,),
    # "clf__max_features" : ("sqrt", "log2", None)
}

# Find the best parameters for both the feature extraction and the
# classifier
grid_search = GridSearchCV(pipeline, parameters, cv=10, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(df[0],df[1])
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
RandomForestClassifierModel = Pipeline([
    ('bow',CountVectorizer(max_df=0.5, max_features=10000)),
    ('tfidf',TfidfTransformer(norm="l2", use_idf=False)),
    ('classifier', RandomForestClassifier(criterion="gini", min_samples_leaf=1, n_estimators=100))
])

In [ ]:
model = RandomForestClassifierModel.fit(df[0],df[1])

In [ ]:
def metin_onisle(metin):
    kucuk_harfli_metin = unicode_tr(metin).lower()
    istenen_karakterler = set(list(' abcdefghijklmnopqrstuvwxyzâçîöüğış0123456789'))
    harfler = list(kucuk_harfli_metin)
    harfler = [k if k in istenen_karakterler else ' ' for k in harfler]
    temiz_dokuman = "".join(kucuk_harfli_metin)
    temiz_dokuman_kelimeleri = temiz_dokuman.split(' ')
    temiz_dokuman_kelimeleri = [kelime for kelime in temiz_dokuman_kelimeleri if len(kelime) > 0]
    stopWords = set(stopwords.words('turkish'))
    temiz_dokuman_kelimeleri = ["" if kelime in stopWords else kelime for kelime in temiz_dokuman_kelimeleri]
    temiz_dokuman_kelimeleri = " ".join(temiz_dokuman_kelimeleri).replace("  ", " ").split()
    turkStem=TurkishStemmer()
    temiz_dokuman_kelimeleri = [turkStem.stemWord(kelime) for kelime in temiz_dokuman_kelimeleri]
    n = 2
    bigrams = ngrams(temiz_dokuman_kelimeleri, n)
    bigramstr = map(''.join, bigrams)
    ngram = " ".join(list(bigramstr))
    temiz_dokuman = " ".join(temiz_dokuman_kelimeleri) + " " +  "".join(ngram)
    return temiz_dokuman

In [ ]:
model.predict([metin_onisle("BaşkentKart")])

In [ ]:
n = 3
probs = model.predict_proba(["BaşkentKart"])
sorted( zip( RandomForestClassifierModel.classes_, probs[0] ), key=lambda x:x[1] )[-n:]

In [ ]:
RandomForestClassifierModel.predict_proba([metin_onisle("Teleferik")])[0]

In [ ]:
RandomForestClassifierModel.predict([metin_onisle("Teleferik")])[0]

In [ ]:
RandomForestClassifierModel.predict([metin_onisle("Başkent Kartın Aidat Ücreti")])[0]

In [ ]:
RandomForestClassifierModel.predict([metin_onisle("Cenaze / defin işlemi")])[0]

In [ ]:
RandomForestClassifierModel.predict([metin_onisle("Öğrencilerin ücretsitaşındığ hat")])[0]

In [ ]:
import pickle

with open('../ankarabbHibritChatbot/RandomForestClassifierModel.pickle', 'wb') as handle:
    pickle.dump(RandomForestClassifierModel, handle, protocol=pickle.HIGHEST_PROTOCOL)
    